In [1]:
import tensorflow as tf
from model_trainer import *
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5


In [2]:
import cv2
import numpy as np
from pathlib import Path
from tqdm.notebook import trange

In [3]:
import backbone_models
import specific_models

In [4]:
import skimage

In [5]:
vid_dir = Path('data/test')
vid_path = vid_dir.iterdir().__next__()

model_path = 'savedmodels/mobv3_small_head/100'
bb_model = backbone_models.mobv3_small
sp_model = {
    'head' : specific_models.conv_squeeze_double,
}
image_size = (384,256)

In [6]:
testmodel = ChaserModel(tf.keras.Input((image_size[1],image_size[0],3)),bb_model,sp_model)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 384, 3)]     0         
_________________________________________________________________
EffHRNet (EfficientHRNet_MV3 (None, 128, 192, 12)      1752632   
_________________________________________________________________
final_conv_head (Conv2D)     (None, 128, 192, 1)       13        
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 256, 384, 1)       0         
_________________________________________________________________
tf.compat.v1.squeeze (TFOpLa (None, 256, 384)          0         
_________________________________________________________________
head (Activation)            (None, 256, 384)          0         
Total params: 1,752,645
Trainable params: 1,737,645
Non-trainable params: 15,000
______________________________________________

In [7]:
testmodel.load_weights(model_path)

In [8]:
original_size = (640,480)
original_hw = (original_size[1],original_size[0])
model_size = image_size
model_hw = (model_size[1],model_size[0])
batch_size = 64

In [9]:
cap = cv2.VideoCapture(str(vid_path))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('results/result.mp4',fourcc,30,original_size)
frames = []
original_frames=[]
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        small_frame = cv2.resize(frame, dsize=model_size)[...,2::-1]
        frames.append(small_frame)
        original_frames.append(frame)
    else:
        break
print('loaded')
batch_num = len(frames) // batch_size
count = 0
for i in trange(batch_num):
    output = testmodel.predict_on_batch(np.array(frames[i*batch_size:(i+1)*batch_size]))
    block_hms = output['head']
    block_poses = np.unravel_index(block_hms.reshape((block_hms.shape[0],-1)).argmax(axis=1),block_hms.shape[1:])
    block_poses = np.swapaxes(block_poses,0,1)
    for block_pose, block_hm in zip(block_poses,block_hms):
        new_block = np.multiply(block_pose,np.divide(original_hw,model_hw)).astype(np.int)
        block_rr, block_cc = skimage.draw.disk(new_block,10, shape=original_hw)
        frame = original_frames[count]
        block_hm = cv2.resize(block_hm, dsize=original_size)
        norm_hm = ((block_hm/np.max(block_hm))*255).astype(np.uint8)
        color_hm = cv2.applyColorMap(norm_hm, cv2.COLORMAP_JET)
        
        mixed = cv2.addWeighted(frame, 0.5, color_hm, 0.5, 0.0)
        mixed[block_rr,block_cc] = [0,255,0]
        writer.write(mixed)
        count += 1

cap.release()
writer.release()
print('done')

loaded



done
